In [7]:
import numpy as np
import cv2
import dlib
# 读取图像
#srcImg = cv2.imread("Images/klst.png")
srcImg=cv2.imread("Images/tank.jpg")

img=srcImg.copy()
# Step 1：构造人脸检测器（dlib初始化）
detector = dlib.get_frontal_face_detector()
# Step 2：检测人脸框（使用人脸检测器返回检测到的人脸框）
faces = detector(img, 1)
# Step 3：载入模型（加载预测器）
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

index=0
# Step 4：获取每一张脸的关键点（实现检测）
for face in faces:
    index+=1
    # 检测当前人脸的关键点
    shape=predictor(img, face)

    # 下面对这句代码进行了分解
    # landmarks = np.matrix([[p.x, p.y] for p in shape.parts()])
    position = np.zeros((68, 2), dtype=int)
    # 通过shape.parts()方法遍历关键点
    for i, p in enumerate(shape.parts()): 
        print('face:'+str(index)+' predictor:'+str(i)+' x:'+str(p.x)+' y:'+str(p.y))
        # 将关键点的x,y放入一个2列68行的矩阵中,需要知道的是，人脸的每个特征对应的关键点索引是固定的，比如36-42描述的右眼。
        landmarks = np.matrix([[p.x, p.y]])
        position[i] = (p.x, p.y)
        # Step 5：绘制每一张脸的关键点（绘制shape中的每个点）
        for idx, point in enumerate(landmarks):
            # 当前关键的坐标:[0,1]表示第0行第1个元素
            pos = (point[0, 0], point[0, 1])
            # 针对当前关键点，绘制一个实心圆
            cv2.circle(img, pos, 2, color=(0, 255, 0), thickness=-1)
            # 字体
            font = cv2.FONT_HERSHEY_SIMPLEX
            # 利用cv2.putText输出1-68,索引序号加1，显示时从1开始。
            cv2.putText(img, str(i + 1), pos, font, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
            
# 绘制结果
cv2.imshow("img", img)
cv2.imshow("src", srcImg)
cv2.imwrite("woman.png",img)
cv2.waitKey()
cv2.destroyAllWindows()



face:1 predictor:0 x:86 y:181
face:1 predictor:1 x:87 y:224
face:1 predictor:2 x:92 y:266
face:1 predictor:3 x:98 y:307
face:1 predictor:4 x:110 y:348
face:1 predictor:5 x:132 y:384
face:1 predictor:6 x:167 y:411
face:1 predictor:7 x:205 y:430
face:1 predictor:8 x:247 y:438
face:1 predictor:9 x:290 y:432
face:1 predictor:10 x:329 y:414
face:1 predictor:11 x:365 y:389
face:1 predictor:12 x:388 y:354
face:1 predictor:13 x:402 y:314
face:1 predictor:14 x:409 y:272
face:1 predictor:15 x:416 y:230
face:1 predictor:16 x:419 y:186
face:1 predictor:17 x:122 y:157
face:1 predictor:18 x:146 y:140
face:1 predictor:19 x:176 y:136
face:1 predictor:20 x:207 y:141
face:1 predictor:21 x:235 y:153
face:1 predictor:22 x:279 y:155
face:1 predictor:23 x:306 y:144
face:1 predictor:24 x:336 y:140
face:1 predictor:25 x:365 y:144
face:1 predictor:26 x:388 y:161
face:1 predictor:27 x:257 y:190
face:1 predictor:28 x:256 y:213
face:1 predictor:29 x:255 y:237
face:1 predictor:30 x:254 y:262
face:1 predictor:31 x:

In [15]:
import numpy as np
import dlib
import cv2
import math

# 模型初始化
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 读取图像
srcImg=cv2.imread("Images/girl9.jpg")
image=srcImg.copy()
# 色彩空间转换彩色(BGR)-->灰度（Gray）
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 获取人脸
faces = detector(gray, 3)
# 对检测到的rects，逐个遍历

for face in faces:
    # 针对脸部的关键点进行处理，构成坐标(x,y)形式
    shape = np.matrix([[p.x, p.y] for p in predictor(gray, face).parts()])
    shape=np.array(shape)
    


def big_eye_adjust_fast(src, PointX, PointY, Radius,minor_axis,angle,maskImg,feathering_radius, Strength):                                # strength 代表大眼程度 可取负值
    processed_image = np.zeros(src.shape, np.uint8)
    processed_image = src.copy()
    height = src.shape[0]
    width = src.shape[1]
    PowRadius = Radius * Radius
 
    #maskImg = np.zeros(src.shape[:2], np.uint8)
    #cv2.ellipse(maskImg, (PointX, PointY), (Radius,minor_axis) ,angle,0, 360, (255, 255, 255), -1)  #加以改进 采用椭圆区域作为遮罩，用来替换新的坐标值 不太好用
    #cv2.circle(maskImg, (PointX, PointY), math.ceil(Radius), (255, 255, 255), -1) # 绘制圆形的mask图像 以radius为半径 颜色为（255,255,255）即白色
    cv2.imshow("maskImg", maskImg)
    cv2.waitKey(0)
    mapX = np.vstack([np.arange(width).astype(np.float32).reshape(1, -1)] * height)             # 创建一个二维数组 用来存放这片区域的横坐标
    mapY = np.hstack([np.arange(height).astype(np.float32).reshape(-1, 1)] * width)             # 创建一个二维数组 用来存放这片区域的纵坐标
 
    OffsetX = mapX - PointX
    OffsetY = mapY - PointY
    dis2 = OffsetX * OffsetX + OffsetY * OffsetY
 
    ScaleFactor = 1 - dis2 / PowRadius                                                          
    ScaleFactor = 1 - Strength / 100 * ScaleFactor                                              # 缩放比例因子
    UX = OffsetX * ScaleFactor + PointX                                                         # 映射后的Xd = x0 + k*(X1-X0)= x0 + k* OffsetX
    UY = OffsetY * ScaleFactor + PointY                                                         
    UX[UX < 0] = 0                                                                              
    UX[UX >= width] = width - 1                                                                 # 边缘值处理 将映射后处于区域外的坐标值进行边缘处理
    UY[UY < 0] = 0
    UY[UY >= height] = height - 1
 
    np.copyto(UX, mapX, where=maskImg == 0)                                                     #  替换区域中的值 把mask图像中为0的点 替换为mapX中对应位置的值
    np.copyto(UY, mapY, where=maskImg == 0)                                                         

    processed_image = cv2.remap(src, UX, UY, interpolation=cv2.INTER_AREA,borderMode=cv2.BORDER_REPLICATE)     # 使用双线性插值法
 
    # 创建羽化遮罩
    feathering_mask = np.zeros(src.shape[:2], np.uint8)
    #cv2.ellipse(feathering_mask, (PointX, PointY), (Radius+feathering_radius, minor_axis+feathering_radius), angle, 0, 360, (255, 255, 255), -1)
    feathering_mask = maskImg
    feathering_mask = cv2.GaussianBlur(feathering_mask, (5,5), feathering_radius) / 255
    # 应用羽化效果
    for i in range(3):  # 对于每个颜色通道
        processed_image[:, :, i] = src[:, :, i] * (1 - feathering_mask) + processed_image[:, :, i] * feathering_mask
    return processed_image


processed_image = srcImg.copy()                                                                 # 通过dlib库中的特征点规定  我们认定36和39为左眼  42和45为右眼（镜像）
Radius_left = shape[39][0]-shape[36][0]                             
Radius_right = shape[45][0]-shape[42][0]

minor_axis_Left = shape[37][0]-shape[41][0]
minor_axis_Right = shape[44][0]-shape[47][0]

Left_angle = math.degrees(math.atan((shape[39][1]-shape[36][1])/(shape[39][0]-shape[36][0])))
Right_angle = math.degrees(math.atan((shape[45][1]-shape[42][1])/(shape[45][0]-shape[42][0])))

print("眼睛大小为"+str(max(Radius_left,Radius_right)))
minor_axis = int(max(minor_axis_Left,minor_axis_Right)*1.5)

height = srcImg.shape[0]
width = srcImg.shape[1]

# 假设landmarks是一个包含左眼睛特征点坐标的数组
landmarks_left = np.array([shape[36],shape[37],shape[38],shape[39],shape[40],shape[41]])
landmarks_right = np.array([shape[42],shape[43],shape[44],shape[45],shape[46],shape[47]])
# 创建一个与原始图像大小相同的空白掩模
maskImg_left = np.zeros((height, width), dtype=np.uint8)
maskImg_right = np.zeros((height, width), dtype=np.uint8)
# 在掩模上绘制多边形，255是填充颜色
cv2.fillPoly(maskImg_left, [landmarks_left], 255)
cv2.fillPoly(maskImg_right, [landmarks_right], 255)


PointX_left, PointY_left, Radius_left, Strength_left = int((shape[36][0]+shape[39][0])/2), int((shape[37][1]+shape[41][1])/2), Radius_left,10
PointX_right, PointY_right, Radius_right, Strength_right = int((shape[42][0]+shape[45][0])/2), int((shape[44][1]+shape[47][1])/2),Radius_right,10

#dst = big_eye_adjust_fast(processed_image, PointX_left, PointY_left, Radius_left,minor_axis_Left, Strength_left)[0]
processed_image = big_eye_adjust_fast(processed_image, PointX_left, PointY_left, Radius_left,minor_axis,Left_angle,maskImg_left,2,Strength_left)
processed_image = big_eye_adjust_fast(processed_image, PointX_right, PointY_right, Radius_right,minor_axis,Right_angle,maskImg_right,2,Strength_right)
cv2.imshow("img",srcImg)
cv2.imshow("big.jpg", processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

眼睛大小为86
